# Data Anonymization

In Pega CDH 8.5 and up, it's now possible to record the historical data as seen by the Adaptive Models. See [this academy challenge](https://academy.pega.com/challenge/exporting-historical-data/v4) for reference. This historical data can be further used to experiment with offline models, but also to fine-tune the OOTB Gradient Boosting model. However, sharing this information with Pega can be sensitive as it contains raw predictor data. 

To this end, we provide a simple and transparent script to fully anonimize this dataset.

The DataAnonymization script is now part of pdstools, and you can import it directly as such.

In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import os  
import sys
import plotly.io as pio
pio.renderers.default = "notebook_connected"

sys.path.append("../../../")
sys.path.append('../../python')

In [2]:
from pdstools import ADMDatamart
from pdstools import Config, DataAnonymization
import polars as pl

## Input data

To demonstrate this process, we're going to anonymise this toy example dataframe:

In [3]:
pl.read_ndjson('../../../../data/SampleHDS.json')

Context_Name,Customer_MaritalStatus,Customer_CLV,Customer_City,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,i64,str,str,str
"""FirstMortgage30yr""","""Married""",1460,"""Port Raoul""","""Account""","""Rejected"""
"""FirstMortgage30yr""","""Unknown""",669,"""Laurianneshire""","""AutoLoans""","""Accepted"""
"""MoneyMarketSavingsAccount""","""No Resp+""",1174,"""Jacobshaven""","""Account""","""Rejected"""
"""BasicChecking""","""Unknown""",1476,"""Lindton""","""Account""","""Rejected"""
"""BasicChecking""","""Married""",1211,"""South Jimmieshire""","""DepositAccounts""","""Accepted"""
"""UPlusFinPersonal""","""No Resp+""",533,"""Bergeville""",null,"""Rejected"""
"""BasicChecking""","""No Resp+""",555,"""Willyville""","""DepositAccounts""","""Rejected"""


As you can see, this dataset consists of regular predictors, IH predictors, context keys and the outcome column. Additionally, some columns are numeric, others are strings. Let's first initialize the DataAnonymization class.

In [4]:
anon = DataAnonymization(hds_folder='../../../../data/')

By default, the class applies a set of anonymisation techniques:
- Column names are remapped to a non-descriptive name
- Categorical values are hashed with a random seed
- Numerical values are normalized between 0 and 1
- Outcomes are mapped to a binary outcome.

To apply these techniques, simply call `.process()`:

In [5]:
anon.process()

PREDICTOR_0,PREDICTOR_1,filename,PREDICTOR_3,Context_Name,IH_PREDICTOR_0,Decision_Outcome
f64,str,str,str,str,str,bool
5.3431e18,"""10831486870363456856""","""../../../../data/SampleHDS.jso…","""12719386505136887422""","""11890007023989276111""","""10868400715796717920""",false
5.5617e18,"""8527985881567514673""","""../../../../data/SampleHDS.jso…","""14052183327114255698""","""11890007023989276111""","""5997451788758191148""",true
1.5859e18,"""124424036889564711""","""../../../../data/SampleHDS.jso…","""16944238344783374646""","""158550944716944236""","""10868400715796717920""",false
1.3431e19,"""8527985881567514673""","""../../../../data/SampleHDS.jso…","""3901875729476741695""","""6653811494134817397""","""10868400715796717920""",false
3.4456e18,"""10831486870363456856""","""../../../../data/SampleHDS.jso…","""5844344389285945587""","""6653811494134817397""","""12561036712258206169""",true
9.1390e18,"""124424036889564711""","""../../../../data/SampleHDS.jso…","""5648729090705388398""","""15811845728705189266""",null,false
1.7275e19,"""124424036889564711""","""../../../../data/SampleHDS.jso…","""15331866792890720054""","""6653811494134817397""","""12561036712258206169""",false


To trace back the columns to their original names, the class also contains a mapping, which does not have to be provided.

In [6]:
anon.column_mapping

{'Customer_CLV': 'PREDICTOR_0',
 'Customer_MaritalStatus': 'PREDICTOR_1',
 'filename': 'filename',
 'Customer_City': 'PREDICTOR_3',
 'Context_Name': 'Context_Name',
 'IH_Web_Inbound_Accepted_pxLastGroupID': 'IH_PREDICTOR_0',
 'Decision_Outcome': 'Decision_Outcome'}

## Configs

Each capability can optionally be turned off - see below for the full list of config options, and refer to the API reference for the full description.

In [7]:
dict(zip(Config.__init__.__code__.co_varnames[1:], Config.__init__.__defaults__))

{'config_file': None,
 'hds_folder': '.',
 'use_datamart': False,
 'datamart_folder': 'datamart',
 'output_format': 'ndjson',
 'output_folder': 'output',
 'mapping_file': 'mapping.map',
 'mask_predictor_names': True,
 'mask_context_key_names': False,
 'mask_ih_names': True,
 'mask_outcome_name': False,
 'mask_predictor_values': True,
 'mask_context_key_values': True,
 'mask_ih_values': True,
 'mask_outcome_values': True,
 'context_key_label': 'Context_*',
 'ih_label': 'IH_*',
 'outcome_column': 'Decision_Outcome',
 'positive_outcomes': ['Accepted', 'Clicked'],
 'negative_outcomes': ['Rejected', 'Impression'],
 'special_predictors': ['Decision_DecisionTime',
  'Decision_OutcomeTime',
  'Decision_Rank'],
 'sample_percentage_schema_inferencing': 0.01}

It's easy to change these parameters by just passing the keyword arguments. In the following example, we
- Keep the IH predictor names
- Keep the outcome values
- Keep the context key values
- Keep the context key predictor names

In [8]:
anon = DataAnonymization(
    hds_folder="../../../../data/",
    mask_ih_names=False,
    mask_outcome_values=False,
    mask_context_key_values=False,
    mask_context_key_names=False,
)
anon.process()


PREDICTOR_0,PREDICTOR_1,filename,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str,str
4.1575e18,"""3149480548280037146""","""../../../../data/SampleHDS.jso…","""12542311172466509671""","""FirstMortgage30yr""","""7537064975952712641""","""Rejected"""
1.0652e19,"""11310407480535828497""","""../../../../data/SampleHDS.jso…","""18316053974807194591""","""FirstMortgage30yr""","""7654035917377455166""","""Accepted"""
5.5470e18,"""1356614725612954196""","""../../../../data/SampleHDS.jso…","""14844646775568122703""","""MoneyMarketSavingsAccount""","""7537064975952712641""","""Rejected"""
4.6315e18,"""11310407480535828497""","""../../../../data/SampleHDS.jso…","""9845140509788936661""","""BasicChecking""","""7537064975952712641""","""Rejected"""
1.2390e19,"""3149480548280037146""","""../../../../data/SampleHDS.jso…","""6334864336833586530""","""BasicChecking""","""6356063117451043038""","""Accepted"""
1.4595e19,"""1356614725612954196""","""../../../../data/SampleHDS.jso…","""12273737014632098359""","""UPlusFinPersonal""",null,"""Rejected"""
1.0791e19,"""1356614725612954196""","""../../../../data/SampleHDS.jso…","""16925345986892011844""","""BasicChecking""","""6356063117451043038""","""Rejected"""


The configs can also be written and read as such:

In [9]:
anon.config.save_to_config_file('config.json')

In [10]:
anon = DataAnonymization(config=Config(config_file='config.json'))
anon.process()

PREDICTOR_0,PREDICTOR_1,filename,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str,str
7.0761e18,"""2361789810396253517""","""../../../../data/SampleHDS.jso…","""16686285685172645345""","""FirstMortgage30yr""","""17004409338086296404""","""Rejected"""
1.5205e19,"""11018414291457401949""","""../../../../data/SampleHDS.jso…","""11817370619844815542""","""FirstMortgage30yr""","""10843666827130830215""","""Accepted"""
1.1905e19,"""18401597918455725878""","""../../../../data/SampleHDS.jso…","""6559103365435694694""","""MoneyMarketSavingsAccount""","""17004409338086296404""","""Rejected"""
1.5686e18,"""11018414291457401949""","""../../../../data/SampleHDS.jso…","""9797982355271122390""","""BasicChecking""","""17004409338086296404""","""Rejected"""
1.0127e18,"""2361789810396253517""","""../../../../data/SampleHDS.jso…","""4586214043502800528""","""BasicChecking""","""18051879944344772158""","""Accepted"""
1.7640e19,"""18401597918455725878""","""../../../../data/SampleHDS.jso…","""14266600082600006176""","""UPlusFinPersonal""",null,"""Rejected"""
6.1390e18,"""18401597918455725878""","""../../../../data/SampleHDS.jso…","""10471941183013278126""","""BasicChecking""","""18051879944344772158""","""Rejected"""


## Exporting
Two functions export:
- `create_mapping_file()` writes the mapping file of the predictor names
- `write_to_output()` writes the processed dataframe to disk

Write to output accepts the following extensions: `["ndjson", "parquet", "arrow", "csv"]`

In [11]:
anon.create_mapping_file()
with open('mapping.map') as f:
    print(f.read())

Customer_CLV=PREDICTOR_0
Customer_MaritalStatus=PREDICTOR_1
filename=filename
Customer_City=PREDICTOR_3
Context_Name=Context_Name
IH_Web_Inbound_Accepted_pxLastGroupID=IH_Web_Inbound_Accepted_pxLastGroupID
Decision_Outcome=Decision_Outcome



In [12]:
anon.write_to_output(ext='arrow')

In [13]:
pl.read_ipc('output/hds.arrow')

PREDICTOR_0,PREDICTOR_1,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str
1.5003e19,"""1870649273708989239""","""6209286911328173376""","""FirstMortgage30yr""","""7122172208037088876""","""Rejected"""
1.2415e19,"""9001570458728140908""","""276917953509087066""","""FirstMortgage30yr""","""2146275036002471067""","""Accepted"""
1.0503e19,"""2843121096913309995""","""1132895760759646637""","""MoneyMarketSavingsAccount""","""7122172208037088876""","""Rejected"""
1.8260e18,"""9001570458728140908""","""17906048586114904262""","""BasicChecking""","""7122172208037088876""","""Rejected"""
3.2928e18,"""1870649273708989239""","""17786880851547946247""","""BasicChecking""","""12475166319505016758""","""Accepted"""
1.4119e19,"""2843121096913309995""","""3763817651507268451""","""UPlusFinPersonal""",null,"""Rejected"""
7.1352e18,"""2843121096913309995""","""1781521713842737451""","""BasicChecking""","""12475166319505016758""","""Rejected"""


## Advanced: Hash fuctions

By default, we use [the same hashing algorithm Polars](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.hash.html#polars.Expr.hash) uses: [xxhash](https://github.com/Cyan4973/xxHash), as implemented [here](https://github.com/pola-rs/polars/blob/3f287f370b3c388ed2f3f218b2c096382548136f/polars/polars-core/src/vector_hasher.rs#L266). xxhash is fast to compute, and you can check its performance in collision, dispersion and randomness [here](https://github.com/Cyan4973/xxHash/tree/dev/tests). 

xxhash accepts four distinct seeds, but by default we set the seeds to `0`. It is possible to set the `seed` argument of the `process()` function to `'random'`, which will set all four seeds to a random integer between `0` and `1000000000`. Alternatively, it is possible to supply the four seeds manually with arguments `seed`, `seed_1`, `seed_2` and `seed_3`. 

If the xxhash with (random) seed(s) is not deemed sufficiently secure, it is possible to use your own hashing algorithm.

Note that since we're now running python code and not native Polars code anymore, this will be _significantly_ slower. Nonetheless, it is possible.

Just as an example - this is how one would use sha3_256:

In [14]:
from hashlib import sha3_256

anon.process(algorithm=lambda x: sha3_256(x.encode()).hexdigest())

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.



ComputeError: AttributeError: 'int' object has no attribute 'encode'